<a href="https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/create_graphic_icecore_start_1700_ce_keelingcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sio-co2o2/keelingcurve_notebooks/main?labpath=notebooks/create_graphic_icecore_start_1700_ce_keelingcurve.ipynb)

# Create a graphic appearing on the Keeling Curve website
### **Graphic: CO<sub>2</sub> record from 1700 to now**

**Mauna Loa, Hawaii CO<sub>2</sub> record starting in 1958 and ice-core CO<sub>2</sub> record before 1958**

#### This notebook creates customizable PDF and PNG images of a graphic appearing on the front page of [keelingcurve.ucsd.edu](keelingcurve.ucsd.edu).

For an overview of the Keeling Curve notebooks, data and graphics licenses, and how to use the notebooks, see [notebooks overview](https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/overview_of_notebooks_keelingcurve.ipynb).

**To use the notebook**, first run all the cells. In Google Colab, select from the top menu and select Runtime -> Run all. Then the buttons will appear to click on which creates and downloads the graphics.

Direct questions to [webmaster-co2o2-sio@ucsd.edu](mailto:webmaster-co2o2-sio@ucsd.edu)

---

### [Go to creating the graphic section](#button-to-create-and-save-the-graphic)

---

### Sample of the graphic produced by this notebook

![title](https://github.com/sio-co2o2/keelingcurve_notebooks/blob/main/images/sample_graphics/co2_1700_ce.png?raw=1?modified=12345678)

## Import packages

In [10]:
import functools
import requests
import re
import pandas as pd
import numpy as np
from scipy import interpolate
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import pathlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import ticker
from matplotlib import rc,rcParams
from matplotlib.path import Path
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.transforms
from matplotlib import animation
from ipywidgets import widgets
from IPython.display import HTML, display
from base64 import b64encode
from PIL import Image
import urllib
import os

%matplotlib inline

In [11]:
# This package is used to convert a vector svg into a png

try:
  from cairosvg import svg2png
except:
  ! pip install cairosvg
  from cairosvg import svg2png

In [12]:
# This import is to enable Google Colab to save files ane then download them

# This import does not exist unless the notebook is run in Google Colab
# Put in a try except block if user wants to use notebook off of Google Colab

try:
    from google.colab import files
except:
    pass

### Set directories and file names

In [13]:
# Get MLO data from the keelingcurve_notebooks github repository
mlo_data_file = 'https://raw.githubusercontent.com/sio-co2o2/keelingcurve_notebooks/main/preliminary_data/mlo/mlo_full_record_now_span.csv'

# Get the icecore dataset going back 2K years
# This is a subset of the full dataset found at https://doi.org/10.25919/5bfe29ff807fb
icecore_2K_data_file = 'https://raw.githubusercontent.com/sio-co2o2/keelingcurve_notebooks/main/icecore_data/Law_Dome_GHG_2000years_co2_by_age.csv'

logo_file = 'https://github.com/sio-co2o2/keelingcurve_notebooks/raw/main/images/logos/ucsd_sio_logo.svg'

plot_dir = pathlib.Path('./plots')
plot_dir.mkdir(exist_ok=True)

plot_basename = 'co2_1700_ce'

pdf_file = plot_dir / f"{plot_basename}.pdf"
png_file = plot_dir / f"{plot_basename}.png"

## **Load in data and prepare it for plotting**

### Load in MLO record

In [14]:
df_mlo = pd.read_csv(mlo_data_file, sep=',', comment='"')
df_mlo.head()

,date,co2,date_seas_adj,co2_seas_adj,date_fit,co2_fit,date_seas_adj_fit,co2_seas_adj_fit,date_filled,co2_filled,date_seas_adj_filled,co2_seas_adj_filled
0,1958.202740,315.71,1958.202740,314.43,1958.202740,316.20,1958.202740,314.91,1958.202740,315.71,1958.202740,314.43
1,1958.287671,317.45,1958.287671,315.16,1958.287671,317.30,1958.287671,314.99,1958.287671,317.45,1958.287671,315.16
2,1958.369863,317.51,1958.369863,314.69,1958.369863,317.88,1958.369863,315.07,1958.369863,317.51,1958.369863,314.69
3,1958.536986,315.87,1958.536986,315.20,1958.454795,317.26,1958.454795,315.15,1958.454795,317.26,1958.454795,315.15
4,1958.621918,314.93,1958.621918,316.21,1958.536986,315.86,1958.536986,315.22,1958.536986,315.87,1958.536986,315.20


### Get MLO Seasonally adjusted data

In [15]:
df_mlo = df_mlo[['date_filled', 'co2_filled']].copy()

### Remove rows with fill values NaN

In [16]:
df_mlo = df_mlo.dropna()

### Convert to numpy arrays to use for labeling relevant years in the plot

In [17]:
mlo_date = df_mlo['date_filled'].to_numpy()
mlo_co2 = df_mlo['co2_filled'].to_numpy()


### **Load in the ice-core record back 2K years**

In [18]:
df_icecore_2K = pd.read_csv(icecore_2K_data_file, sep=',', comment='#', skipinitialspace=True)
df_icecore_2K.head()

,Sample ID,Ice Age (year AD),CO2 Age (year AD),CO2 (ppm),Uncert (ppm)
0,DSSW20K 15.8,firn,1996,359.87,0.02
1,DSSW20K 29,firn,1994,357.24,0.04
2,DE08-2 0,firn,1993,354.60,0.02
3,DSSW20K 37.8,firn,1992,353.89,0.04
4,DE08-2 10,firn,1991,352.60,0.18


### Sort ice-core dataframe by CO2 Age

In [19]:
df_icecore_2K = df_icecore_2K.sort_values(by=['CO2 Age (year AD)'], ascending=True)

df_icecore_2K = df_icecore_2K.reset_index(drop=True)

df_icecore_2K.head()

,Sample ID,Ice Age (year AD),CO2 Age (year AD),CO2 (ppm),Uncert (ppm)
0,DSS 135,94,154,278.2,0.4
1,DSS 134,115,175,277.7,0.4
2,DSS 132,181,241,280.4,0.6
3,DSS 131,206,266,278.4,0.6
4,DSS 082,214,274,280.2,2.5


### Get subset of datafreame and rename columns

In [20]:
df_icecore_2K.drop(['Sample ID', 'Ice Age (year AD)', 'Uncert (ppm)'], axis=1, inplace=True)

df_icecore_2K.columns = ['date_ce', 'co2']

df_icecore_2K.head()

,date_ce,co2
0,154,278.2
1,175,277.7
2,241,280.4
3,266,278.4
4,274,280.2


### Filter ice-core data up to 1957 (So as not to overlap with MLO data)

In [21]:
df_icecore_2K = df_icecore_2K[df_icecore_2K['date_ce'] < 1958]

df_icecore_2K.tail()

,date_ce,co2
264,1956,314.40
265,1957,314.50
266,1957,314.40
267,1957,316.45
268,1957,315.10


### Combine MLO and Icecore data

#### Rename columns of MLO dataframe so can align with the icecore dataframe to concatenate

In [22]:
df_mlo.columns = ['date_ce', 'co2']

#### Verically concatenate the dataframes

In [23]:
df_combined = pd.concat([df_icecore_2K, df_mlo], ignore_index=True)

# And reset the index
df_combined = df_combined.reset_index(drop=True)

df_combined.head()

,date_ce,co2
0,154.0,278.2
1,175.0,277.7
2,241.0,280.4
3,266.0,278.4
4,274.0,280.2


#### Get a subset of the data starting at 1700

We need the data set to stretch back one more data point so that the plot line of the data displays as starting at 1700.

In [24]:
# Find the rows with a date starting on 1700
index_list = df_combined.index[df_combined['date_ce'] >= 1700].tolist()

# Get the first index and then add one more row by appending the min_index - 1
min_index = min(index_list)
index_list = [min_index - 1] + index_list

# Now cull the data to this index list
df_combined = df_combined.iloc[index_list]
df_combined.head()

,date_ce,co2
65,1694.0,277.4
66,1703.0,275.0
67,1710.0,276.3
68,1719.0,276.4
69,1723.0,278.3


#### Convert to numpy arrays for plotting

In [25]:
combined_years = df_combined['date_ce'].to_numpy()
combined_co2 = df_combined['co2'].to_numpy()

## **Define accessory functions**

## Functions to convert dates into various formats
Used for plot limits and labeling

### Function to convert a datetime to a float

In [26]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def dt2t(adatetime):
    """
    Convert adatetime into a float. The integer part of the float should
    represent the year.
    Order should be preserved. If adate<bdate, then d2t(adate)<d2t(bdate)
    time distances should be preserved: If bdate-adate=ddate-cdate then
    dt2t(bdate)-dt2t(adate) = dt2t(ddate)-dt2t(cdate)
    """
    year = adatetime.year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    return year + ((adatetime - boy).total_seconds() / ((eoy - boy).total_seconds()))

### Function to convert a float to a datetime

In [27]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def t2dt(atime):
    """
    Convert atime (a float) to DT.datetime
    This is the inverse of dt2t.
    assert dt2t(t2dt(atime)) == atime
    """
    year = int(atime)
    remainder = atime - year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    seconds = remainder * (eoy - boy).total_seconds()
    return boy + timedelta(seconds=seconds)

### Function to create various date versions for today's date

In [28]:
def get_todays_date_variations():
    
    now = datetime.now()
    todays_decimal =  dt2t(now)
    
    today = date.today()

    todays_day = today.day
    todays_month = today.strftime("%B")
    todays_year = today.year
    todays_date_moyr = today.strftime("%B %Y")
    todays_date_modyyr = f"{todays_month} {todays_day}, {todays_year}"

    return todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year

### Function to get the end date of data

In [29]:
def get_data_end_date(date_data):
    last_date = np.max(date_data)
    last_date_datetime = t2dt(last_date)
    
    # Convert date format to month_name day, year
    day = last_date_datetime.day
    month = last_date_datetime.strftime("%B")
    year = last_date_datetime.year
    date_modyyr = f"{month} {day}, {year}"
    return date_modyyr

<a name="functions-to-modify-graphic"></a>
## **Define functions that can modify the graphic**

### Function to create gradient under curve

In [30]:
# Gradient function modified from the original function in
# the answer from https://stackoverflow.com/questions/29321835/is-it-possible-to-get-color-gradients-under-curve-in-matplotlib

# Modified to add gradient below curve and have a bottom alpha
# The color under the plot line varies as a gradient from the line to the
# bottom axis.

# Modify arguments of the original function to include limits of the 
# gradient under the curve, the transparency and gradient color
# def gradient_fill(x, y, fill_color=None, ax=None, **kwargs):
def gradient_fill(x, y, fill_color='#FFFFFF', xmin=None, ymin=None, xmax=None, 
                  ymax=None, alpha_bottom=None, ax=None, alpha=1, zorder=1, **kwargs):
    """
    Plot a line with a linear alpha gradient filled beneath it.

    Parameters
    ----------
    x, y : array-like
        The data values of the line.
    fill_color : a matplotlib color specifier (string, tuple) or None
        The color for the fill. If None, the color of the line will be used.
    ax : a matplotlib Axes instance
        The axes to plot on. If None, the current pyplot axes will be used.
    Additional arguments are passed on to matplotlib's ``plot`` function.

    Returns
    -------
    line : a Line2D instance
        The line plotted.
    im : an AxesImage instance
        The transparent gradient clipped to just the area beneath the curve.
    """
    if ax is None:
        ax = plt.gca()

    # Only want the gradient to show and not the line defining the gradient shape
    # so change the original function
    #line, = ax.plot(x, y, **kwargs)
    #if fill_color is None:
    #    fill_color = line.get_color()

    if alpha_bottom is None:
        alpha_bottom = 0

    if xmin is None:
        xmin = x.min()

    if ymin is None:
        ymin = y.min()
        
    if xmax is None:
        xmax = x.max()

    if ymax is None:
        ymax = y.max()
        
    if zorder is None:
        zorder = 1
        
    # zorder in alpha of the gradient are now set in the function arguments,
    # so modify this portion of the original function
    #zorder = line.get_zorder()
    
    #alpha = line.get_alpha()
    #alpha = 1.0 if alpha is None else alpha

    z = np.empty((100, 1, 4), dtype=float)

    rgb = mcolors.colorConverter.to_rgb(fill_color)
    z[:, :, :3] = rgb
    #z[:,:,-1] = np.linspace(0, alpha, 100)[:,None]
    z[:, :, -1] = np.linspace(alpha_bottom, alpha, 100)[:, None]

    # Get the gradient limits from the function arguments,
    # so modify the original function
    #xmin, xmax, ymin, ymax = x.min(), x.max(), y.min(), y.max()
    
    im = ax.imshow(z, aspect='auto', extent=[xmin, xmax, ymin, ymax],
                   origin='lower', zorder=zorder)

    xy = np.column_stack([x, y])
    xy = np.vstack([[xmin, ymin], xy, [xmax, ymin], [xmin, ymin]])
    clip_path = Polygon(xy, facecolor='none', edgecolor='none', closed=True)
    ax.add_patch(clip_path)
    im.set_clip_path(clip_path)
    
    ax.autoscale(True)
    
    # Modify original function since there is no need 
    # to return the line and im objects
    #return line, im

### Function to set global fonts and linewidth properties

In [31]:
def set_matplotlib_properties():
    
    # Set default properties for matplotlib
    
    # Reset rcparams in case modified defaults in another notebook during same session 
    plt.rcParams.update(plt.rcParamsDefault)

    plt.rcParams.update({'axes.linewidth':1.5})
    
    plt.rcParams.update({
        "text.usetex": False,
        "font.family": "sans-serif",
        "font.weight":  "normal",
        "font.sans-serif": ["Arial", "Tahoma", "Helvetica","FreeSans", "NimbusSans", "LiberationSans","DejaVu Sans"],
        "mathtext.default":'regular',
        "mathtext.fontset": "dejavusans"
    })
        
    # http://phyletica.org/matplotlib-fonts/
    # This causes matplotlib to use Type 42 (a.k.a. TrueType) fonts 
    # for PostScript and PDF files. This allows you to avoid Type 3 fonts.
    # Turning on usetex also works
    # Needed on Ubuntu for pdf fonts 
    plt.rcParams.update({
      'pdf.fonttype': 42,
        'ps.fonttype': 42 
    })

### Function to set axes properties of the graphic

In [32]:
def set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel):

    # ---------------------------------
    # Plot properties for website plots
    # ---------------------------------

    # Allow room at top for the 2 titles
    fig.subplots_adjust(top=0.85)
    
    ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)

    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=16)

    ax.tick_params(axis='x', which='major', direction='in', length=6, width=1)
    ax.tick_params(axis='y', which='major', direction='in', length=8, width=1)
    
    yaxis_tick_spacing = 50
    ax.yaxis.set_major_locator(ticker.MultipleLocator(yaxis_tick_spacing))

    xaxis_tick_spacing = 50
    ax.xaxis.set_major_locator(ticker.MultipleLocator(xaxis_tick_spacing))

    # Don't display first year since runs into y axis numbers
    xticks_locs = ax.get_xticks()
    xtick_labels = ax.xaxis.get_ticklabels()
    for index, loc in enumerate(xticks_locs):
        if loc == xmin:
            xtick_labels[index].set_visible(False)
    
    ax.tick_params(which='minor', direction='in', length=5)
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    
    labels = ax.get_xticklabels() + ax.get_yticklabels()
    for label in labels: label.set_fontweight('bold')

    ax.set_xlabel(xlabel, fontweight='bold', fontsize=12, labelpad=5)
    ax.set_ylabel(ylabel, fontweight='bold', fontsize=18, labelpad=5)

    # Set axes limits last 
    # If did before, setting xtick labels past xmin & xmax would have
    # extended the plot limits
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

### Function to place the titles

In [33]:
def add_plot_title(ax, title1, title2):

    ax.annotate(title1, xy=(0, 1.15), xycoords='axes fraction', fontsize=14,
                horizontalalignment='left', verticalalignment='top', fontweight="normal")
    ax.annotate(title2, xy=(0, 1.07), xycoords='axes fraction', fontsize=18,
                horizontalalignment='left', verticalalignment='top', fontweight = 'bold')

### Function to add an inset label

In [34]:
def add_inset_label(ax, label_text):
    ax.annotate(label_text, xy=(0.03, 0.9), xycoords='axes fraction', fontsize=12,
                horizontalalignment='left', verticalalignment='top')


### Function to save the graphic

In [35]:
def save_graphic(fig, pdf_file, png_file):

    fig_width, fig_height = fig.get_size_inches()

    # For pdf
    width_in = 10
    height_in = (fig_height/fig_width) * width_in

    fig.set_size_inches(width_in, height_in)

    # Save to a high dpi so that logo png file has a high resolution
    fig.savefig(pdf_file, facecolor='w', edgecolor='w',
                orientation='landscape', format=None,
                transparent=False, bbox_inches='tight', dpi=600)

    # For png
    # Want a png of height 500px

    # First save figure as a png and then scale to size needed

    png_dpi = 300

    fig.savefig(png_file, facecolor='w', edgecolor='w',
                orientation='landscape', dpi=png_dpi, bbox_inches='tight')

    # Now resize the image to be width 1000px for the
    # keelingcurve website
    img = Image.open(png_file)
    img_width, img_height = img.size

    # width_px = 1000
    # img_scale = (width_px / img_width)
    # height_px = int((float(img_height) * float(img_scale)))

    height_px = 500
    img_scale = (height_px / img_height)
    width_px = int((float(img_width) * float(img_scale)))

    #img = img.resize((width_px, height_px), Image.Resampling.LANCZOS)
    img = img.resize((width_px, height_px), Image.ANTIALIAS)
    img.save(png_file)


### Function to create a Context Manager

This allows a message to display in a cell while a function is running.
Usefull for indicating a function is running.

In [36]:
class ShowProgressContextManagerWidgets:

    def __init__(self, out, command=''):
        self.out = out
        self.command = command
        
    def __enter__(self):
        css_style = "<style>span.start { margin:0; padding: 0; color: blue; }</style>"
        html_str = f'<span class="start">Running {self.command}...</span>'

        self.out.clear_output()
        
        with self.out:
            display(HTML(css_style))
            display(HTML(html_str))

        
    def __exit__(self, exc_type, exc_value, exc_tb):
        
        css_style = "<style>span.end { margin: 0; padding: 0; color: green; }</style>"
        html_str = '<span class="end">Finished</span>'

        self.out.clear_output()

        with self.out:
            display(HTML(css_style)) 
            display(HTML(html_str))


# **Steps to create the graphic**

### Change any properties in the section [Functions to modify the graphic](#functions-to-modify-graphic)

### Comment out any function in the overall function [create_graphic](#function-to-create-the-graphic) to remove an element from the final graphic

Comment out a line by adding a # symbol in front of a statement


## **Set properties to use for matplotlib (plotting engine)**

In [37]:
set_matplotlib_properties()

## **Set plot limits and labeling**

#### Get todays date in various formats for labeling|

In [38]:
todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year = get_todays_date_variations()

### Set titles and axes labels

In [39]:
xlabel = 'Years (C.E.)'
ylabel = "$\mathregular{CO}\\bf{_2}$" + " Concentration (ppm)"

title1 = ""
title2 = "Mauna Loa data starting in 1958. Ice-core data before 1958."

### Set yaxis min and max limits

In [40]:
ymin = 200
ymax = 430

### Set xaxis min and max limits

In [41]:
xmin = 1700
xmax = todays_decimal

<a name="function-to-create-the-graphic"></a>
## **Function to create the graphic**

In [42]:
def create_graphic(msg_out, graphic_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, combined_years, combined_co2, btn):

    msg_out.clear_output()
    graphic_out.clear_output()

    # create a figure container
    fig = plt.figure(figsize=(10, 7))

    # Set axes
    # Add padding for exterior plot text
    ax = plt.axes([0.11, 0.12, 0.83, 0.54])

    # ---------------------------------------------------
    # Context manager 
    # 
    # Used to show messages while creating the animiation
    # ---------------------------------------------------

    with ShowProgressContextManagerWidgets(msg_out, 'Create Graphic') as progress:

        # ------------------------
        # Add gradient under curve
        # ------------------------

        # The gradient is a guide to the eye

        # Set gradient limits to the data limits
        data_start_date = min(combined_years)
        data_end_date = max(combined_years)

        area_color = '#73a7e6'

        gradient_fill(combined_years, combined_co2, fill_color=area_color, 
                    xmin=data_start_date, ymin=ymin, xmax=data_end_date, 
                    ymax=ymax, ax=ax, alpha_bottom=0.1, alpha=1.0)

        # -------------
        # Plot the data
        # -------------

        # Set plotting zorder above the gradientZ
        ax.plot(combined_years, combined_co2, '-', color='black', linewidth=0.75, zorder=5)

        # -------------------
        # Set plot properties
        # -------------------
        set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel)

        # ---------------
        # Add plot titles
        # ---------------
        add_plot_title(ax, title1, title2)

        # --------------
        # Add inset text
        # --------------
        inset_text = f'Last updated {todays_date_modyyr}'
        add_inset_label(ax, inset_text)

        # ------------
        # Save graphic
        # ------------
        fig = plt.gcf()

        save_graphic(fig, pdf_file, png_file)

    with graphic_out:

        plt.show()

<a name='button-to-create-and-save-the-graphic'></a>
## **Button to create and save the graphic**

In [43]:
msg_out = widgets.Output()
graphic_out = widgets.Output()

button = widgets.Button(description='Create Graphic', button_style='primary')
button.on_click(functools.partial(create_graphic, msg_out, graphic_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, combined_years, combined_co2))
display(button)

display(msg_out)
display(graphic_out)

Button(button_style='primary', description='Create Graphic', style=ButtonStyle())

Output()

Output()

## **Button to download PDF & PNG files of the graphic**

The download button only works when the notebook is run with Google Colab

If the notebook is run on a local machine, the graphics are inside the plots directory

In [44]:
def download_files(msg_out, btn):

    msg_out.clear_output()

    try:
        files.download(png_file)
        files.download(pdf_file)
    except:
        with msg_out:
            print("If running notebook in Google Colab:")
            print("Need to create the graphic first.")
            print(f"\nIf running in a python environment, the graphics are in the plots directory.")


msg_out = widgets.Output()

button = widgets.Button(description="Download Graphic Files", button_style="primary")

layout = widgets.Layout(width='auto', height='auto') #set width and height

button = widgets.Button(
    description='Download Graphic Files',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    button_style='primary',
    layout = layout
)   

button.on_click(functools.partial(download_files, msg_out))

display(button)
display(msg_out)

Button(button_style='primary', description='Download Graphic Files', layout=Layout(height='auto', width='auto'…

Output()